# Using QEPy with Envyron
## Simulations of Isolated Systems in a Surface-Tension Medium

In [ ]:
import qepy
import importlib

In [ ]:
import numpy as np
from qepy.driver import Driver
from qepy.io import QEInput

In [ ]:
from ase import Atoms
import matplotlib.pyplot as plt

## Set System and QE Parameters

In [ ]:
qe_options = {
    '&control': {
        'calculation': "'scf'",
        'pseudo_dir': "'./data/pseudo/'"
    },
    '&system': {
        'ecutrho' : 150,
        'ecutwfc' : 30,
        'ibrav' : 0,
        'nat' : 3,
        'ntyp' : 2,
        'ibrav' : 0
    },
    '&electrons': {
        'conv_thr' : 1e-10,
        'diagonalization' : "'cg'",
        'mixing_beta' : 0.4,
        'electron_maxstep' : 200
    },
    'atomic_positions bohr': ['O   6.79  7.05  6.50','H   8.45  6.22  6.50','H   5.56  5.66  6.50'],
    'atomic_species': ['H   1.  H.pbe-rrkjus.UPF','O  16.  O.pbe-rrkjus.UPF'],
    'k_points automatic': ['1 1 1 0 0 0'],
    'cell_parameters bohr':[
        '15.  0.    0.',
        '0.  15.    0.',
        '0.   0.   15.'],
}

In [ ]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.out', ldscf=True)

Run the SCF loop in the notebook

In [ ]:
for i in range(60):
    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('Iter: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break
driver.calc_energy()

## Setup Environment and Calculator

Extract ASE atoms from the QEPy driver

In [ ]:
atoms = driver.get_ase_atoms()

Convert ASE atoms data into the quantities expected by Environ

NOTE: valence charges need to be extracted from the QEpy driver

In [ ]:
natoms = len(atoms.numbers)
ntypes = len(np.unique(atoms.numbers))
ion_ids = list(np.unique(atoms.numbers))
ion_labels = list(np.unique(atoms.get_chemical_symbols()))
ion_weigths = list(np.unique(atoms.get_masses()))
itypes = [ ion_ids.index(id) for id in atoms.numbers]
zv = list(driver.qepy_modules.ions_base.get_array_zv()[:ntypes])
coords = atoms.positions / 0.52917720859

Generate an Environ grid extracting information on the cell and grid from the driver

In [ ]:
from envyron.domains import EnvironGrid
at = driver.get_ions_lattice()
nr = driver.get_number_of_grid_points()
grid = EnvironGrid(at, nr, label='system')

In [ ]:
from envyron.representations import EnvironDensity
rho = EnvironDensity(grid)

Read Environ input

In [ ]:
from envyron.io.input import Input
my_input = Input(natoms=natoms, filename='data/surface.yml')

Create the Environ Setup 

In [ ]:
from envyron.setup import Setup
my_setup = Setup(my_input)
my_setup.init_cell(grid)
my_setup.init_numerical(False)

Create the Environ Object

In [ ]:
from envyron.main import Main
environ = Main(my_setup,natoms,ntypes,itypes,zv,ion_ids)
environ.update_cell_dependent_quantities()
environ.update_ions(coords)

Setup the Environ Calculator

In [ ]:
from envyron.calculator import Calculator
my_calculator = Calculator(environ)

## Run the Calculation with Environ

Restart a new driver

In [ ]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.out', ldescf=True)

In [ ]:
rho = driver.data2field(driver.get_density().copy())
environ.update_electrons(rho)
surfaces = [environ.solvent.surface]
volumes = [environ.solvent.volume]

In [ ]:
for i in range(60):

    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('Iter: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break

    # compute Environ contribution to the energy
    my_calculator.energy()
    # pass new electronic density to Environ
    rho = driver.data2field(driver.get_density().copy())
    environ.update_electrons(rho)
    # compute Environ contribution to the potential 
    my_calculator.potential(True)

    # saves volumes
    surfaces.append(environ.solvent.surface)
    volumes.append(environ.solvent.volume)

    driver.embed.extene = environ.esurface
    driver.set_external_potential(driver.field2data(environ.vsoftcavity), exttype=0)

etot = driver.get_energy()
my_calculator.energy()
print(etot, environ.esurface, etot + environ.esurface)
etot = driver.calc_energy()

In [ ]:
environ.vsoftcavity.integral()

In [ ]:
reference_surfaces_chain = [420.2466823, 324.7721826, 220.7000072, 179.6244348, 182.3205339, 182.3622363, 182.6495202, 183.1627543, 183.2637461, 183.3127414, 183.2923328, 183.3210003, 183.3252566, 183.3491582, 183.3266997, 183.3353964, 183.3353964]
reference_surfaces_fft = [357.0177518, 289.7736811, 240.0993095, 212.9943399, 216.0989554, 215.6295828, 215.9762042, 216.3968949, 216.4501114, 216.4470844, 216.4719404, 216.4658392, 216.4648545, 216.4635649, 216.4629532, 216.4545856, 216.4526368, 216.4648408, 216.4863305, 216.4867332, 216.4766621, 216.4741522, 216.4764870, 216.4927295, 216.4951850, 216.4967752, 216.4895422, 216.4902171, 216.4871166, 216.4869689, 216.4849923, 216.4878092, 216.4873259, 216.4855476, 216.4832269, 216.4932361, 216.4990557, 216.4949343, 216.4797052, 216.4754609, 216.4703574, 216.4689084, 216.4793384, 216.4749973, 216.4748600, 216.4819177, 216.4696064, 216.4917723, 216.4928425, 216.4928270, 216.4942967, 216.4929711, 216.4859061, 216.4851681, 216.4800907, 216.4806906, 216.4828521, 216.4842396, 216.4835889, 216.4836822, 216.4834928, 216.4881784, 216.4881784]

In [ ]:
plt.plot(reference_surfaces_fft,'o-')
plt.plot(surfaces,'o:')
#plt.ylim(210,220)
#plt.ylim(182,187)

In [ ]:
surfaces